# **Transfer Learning for Agricultural Images Classification**

## **Import Libraries**

In [1]:
import tensorflow as tf
from tensorflow.keras.applications import MobileNet
from tensorflow.keras.preprocessing.image import ImageDataGenerator
import matplotlib.pyplot as plt
import numpy as np
from zipfile import ZipFile

In [2]:
!kaggle datasets download mdwaquarazam/agricultural-crops-image-classification

'kaggle' is not recognized as an internal or external command,
operable program or batch file.


## **Import Dataset**

In [3]:
with ZipFile('agricultural-crops-image-classification.zip') as zip :
  zip.extractall()

FileNotFoundError: [Errno 2] No such file or directory: 'agricultural-crops-image-classification.zip'

## **Data Preprocessing**

In [ ]:
DATA_DIR = 'Agricultural-crops'

In [ ]:
train_datagen = ImageDataGenerator(rescale=1/255,
                                   width_shift_range=0.05,
                                   height_shift_range=0.05,
                                   rotation_range=5,
                                   horizontal_flip=True,
                                   validation_split=0.1)
test_datagen = ImageDataGenerator(rescale=1/255, validation_split=0.1)

In [ ]:
train_gen = train_datagen.flow_from_directory(DATA_DIR,
                                              target_size=(224,224),
                                              subset='training',
                                              batch_size=32,
                                              class_mode='categorical')

test_gen = test_datagen.flow_from_directory(DATA_DIR,
                                            target_size=(224,224),
                                            subset='validation',
                                            batch_size=32,
                                            class_mode='categorical',
                                            shuffle=False)

In [ ]:
train_batch1 = next(train_gen)
train_images_batch1 = train_batch1[0]
train_labels_batch1 = train_batch1[1]

test_batch1 = next(test_gen)
test_images_batch1 = test_batch1[0]
test_labels_batch1 = test_batch1[1]

In [ ]:
train_indices = train_gen.class_indices
train_labels_dict = {value: key for key, value in train_indices.items()}
test_indices = test_gen.class_indices
test_labels_dict = {value: key for key, value in test_indices.items()}

## **Visualizing Dataset**

In [ ]:
np.random.seed(88888)
idx = np.random.choice(train_images_batch1.shape[0], size=25, replace=False)
selected_imgs = [train_images_batch1[i] for i in idx]
selected_labels = [np.argmax(train_labels_batch1[i]) for i in idx]
plt.figure(figsize=(18,18))
for i in range(25) :
  plt.subplot(5, 5, i+1)
  plt.imshow(selected_imgs[i])
  plt.title(train_labels_dict[selected_labels[i]], fontsize=18)
  plt.axis(False)
plt.tight_layout(pad=0.8)
plt.show()

## **Import MobileNet Model**

In [ ]:
mobilenet = MobileNet(input_shape=(224, 224, 3), include_top=False, weights='imagenet')

In [ ]:
mobilenet.summary()

## **Modelling**

In [ ]:
tf.keras.backend.clear_session()

In [ ]:
for layer in mobilenet.layers :
  layer.trainable = False

In [ ]:
mobilenet_output = mobilenet.output

In [ ]:
x = tf.keras.layers.Conv2D(filters=128, kernel_size=(3,3), padding='same', activation='relu')(mobilenet_output)
x = tf.keras.layers.MaxPooling2D(pool_size=(2,2))(x)
x = tf.keras.layers.BatchNormalization()(x)
x = tf.keras.layers.GlobalAveragePooling2D()(x)
x = tf.keras.layers.Dropout(0.5)(x)
x = tf.keras.layers.Dense(128, activation='relu')(x)
x = tf.keras.layers.Dropout(0.5)(x)
output = tf.keras.layers.Dense(30, activation='softmax')(x)

model = tf.keras.Model(inputs=mobilenet.input, outputs=output)

In [ ]:
model.summary()

## **Training the Model**

In [ ]:
model.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=1e-3),
              loss=tf.keras.losses.CategoricalCrossentropy(),
              metrics=['accuracy'])

In [ ]:
reduceLR = tf.keras.callbacks.ReduceLROnPlateau(monitor="val_accuracy", patience=3, factor=0.9)

In [ ]:
class stopEarly(tf.keras.callbacks.Callback) :
    def on_epoch_end(self, epoch, logs=None) :
        if logs.get('accuracy') >= 0.9 and logs.get('val_accuracy') >= 0.8 :
            self.model.stop_training = True

stop_early = stopEarly()

In [ ]:
history = model.fit(
    train_gen,
    validation_data=test_gen,
    epochs=50,
    callbacks=[reduceLR, stop_early]
)

## **Model Evaluation**

In [ ]:
test_loss, test_accuracy = model.evaluate(test_gen)
print(f"Test Accuracy: {test_accuracy:.4f}")

In [ ]:
plt.figure(figsize=(18,8))

plt.subplot(121)
plt.title('Accuracy')
plt.plot(history.history['accuracy'], label='Training')
plt.plot(history.history['val_accuracy'], label='Testing')
plt.legend()

plt.subplot(122)
plt.title('Loss')
plt.plot(history.history['loss'], label='Training')
plt.plot(history.history['val_loss'], label='Testing')
plt.legend()

plt.show()

## **Visualizing Predictions**

In [ ]:
predictions = model.predict(test_images_batch1)
predicted_labels = np.argmax(predictions, axis=-1)
true_labels = np.argmax(test_labels_batch1, axis=-1)

np.random.seed(88888)
idx = np.random.choice(len(test_images_batch1), size=25, replace=False)
selected_test_imgs = [test_images_batch1[i] for i in idx]
selected_predicted_labels = [predicted_labels[i] for i in idx]
selected_true_labels = [true_labels[i] for i in idx]

plt.figure(figsize=(18, 18))
for i in range(25):
    plt.subplot(5, 5, i+1)
    plt.imshow(selected_test_imgs[i])
    plt.title(f"Pred: {test_labels_dict[selected_predicted_labels[i]]}\nTrue: {test_labels_dict[selected_true_labels[i]]}", fontsize=18)
    plt.axis(False)
plt.tight_layout(pad=0.8)
plt.show()

In [ ]:
img = plt.imread('/kaggle/input/agricultural-crops-image-classification/Agricultural-crops/coconut/image (1).jpeg')
plt.imshow(img)

In [ ]:
from PIL import Image
import numpy as np

from PIL import Image
import numpy as np

def preprocess_image(image):
    # Convert NumPy array to PIL Image if necessary
    if isinstance(image, np.ndarray):
        image = Image.fromarray(image)

    image = image.convert("RGB")  # Ensure 3 color channels
    image = image.resize((224, 224))  # Resize to match model's expected input
    image = np.array(image) / 255.0  # Normalize pixel values
    image = np.expand_dims(image, axis=0)  # Add batch dimension
    return image



processed_image = preprocess_image(img)

# Predict
prediction = model.predict(processed_image)
predicted_class = np.argmax(prediction)

print(f"Predicted Class: {predicted_class}")

In [ ]:
# # train_indices = train_gen.class_indices
# train_labels_dict = {value: key for key, value in train_indices.items()}

In [ ]:
train_labels_dict